In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
file01_df = pd.read_excel('file_01_current_suppy_network.xlsb', engine='pyxlsb')
file02_df = pd.read_excel('file_02_cost_data.xlsb', engine='pyxlsb')
file04_df = pd.read_excel('file_04_zone_info.xlsb', engine='pyxlsb')

In [4]:
file01_df.head()

,supplier_location,customer_location,shpment_size,payment_option,product_quantity,current_logistics_vendor
0,facility_01,201301,small,cod,2156,v1
1,facility_01,122001,small,cod,1859,v1
2,facility_01,248001,small,cod,1805,v1
3,facility_01,201301,small,prepaid,1555,v1
4,facility_01,800001,small,cod,1508,v1


In [5]:
file02_df.head()

,zone,mode,size,cost_with_vendor_v1,cost_with_vendor_v2,cost_with_vendor_v3,cost_with_vendor_v4
0,zone_C,cod,large,1480,1480,1100,1080
1,zone_C,cod,medium,480,560,410,410
2,zone_C,cod,small,90,120,80,80
3,zone_C,prepaid,large,1450,1430,1040,1050
4,zone_C,prepaid,medium,450,510,380,370


In [6]:
file04_df.head()

,source,destination,shipment_zone
0,facility_06,110001,zone_a
1,facility_06,110001,zone_a
2,facility_06,110001,zone_a
3,facility_06,110001,zone_a
4,facility_06,110001,zone_a


In [21]:
q01_pivot = pd.pivot_table(file01_df, index=['current_logistics_vendor'], values='product_quantity', aggfunc=np.sum)
q01_pivot['% Share'] = q01_pivot['product_quantity']*100/q01_pivot['product_quantity'].sum()
q01_pivot

,product_quantity,% Share
current_logistics_vendor,,
v1,3099255,88.416936
v2,94423,2.693742
v3,116688,3.328928
v4,194907,5.560394


In [85]:
# Q-01
q01_pivot = pd.pivot_table(file01_df, index=['current_logistics_vendor'], values='product_quantity', aggfunc=np.sum)
q01_pivot['% share'] = q01_pivot['product_quantity']*100/q01_pivot['product_quantity'].sum()
display(q01_pivot)

,product_quantity,% share
current_logistics_vendor,,
v1,3099255,88.416936
v2,94423,2.693742
v3,116688,3.328928
v4,194907,5.560394


In [29]:
file02_long = pd.melt(file02_df, id_vars=['zone','mode', 'size'], value_vars=['cost_with_vendor_v1', 'cost_with_vendor_v2',
       'cost_with_vendor_v3', 'cost_with_vendor_v4'], value_name='price', var_name ='vendor')
file02_long['vendor'] = file02_long['vendor'].str.replace('cost_with_vendor_', '')
file02_long

,zone,mode,size,vendor,price
0,zone_C,cod,large,v1,1480
1,zone_C,cod,medium,v1,480
2,zone_C,cod,small,v1,90
3,zone_C,prepaid,large,v1,1450
4,zone_C,prepaid,medium,v1,450
...,...,...,...,...,...
67,Zone_B,cod,medium,v4,320
68,Zone_B,cod,small,v4,70
69,Zone_B,prepaid,large,v4,810
70,Zone_B,prepaid,medium,v4,290


In [30]:
file01_w_zone = file01_df.merge(file04_df, left_on = ['supplier_location', 'customer_location'], \
                               right_on = ['source', 'destination'], how = 'left')
cost_df = file01_w_zone.merge(file04_df, left_on =[''] )

,supplier_location,customer_location,shpment_size,payment_option,product_quantity,current_logistics_vendor,source,destination,shipment_zone
0,facility_01,201301,small,cod,2156,v1,facility_01,201301,zone_a
1,facility_01,201301,small,cod,2156,v1,facility_01,201301,zone_a
2,facility_01,201301,small,cod,2156,v1,facility_01,201301,zone_a
3,facility_01,201301,small,cod,2156,v1,facility_01,201301,zone_a
4,facility_01,201301,small,cod,2156,v1,facility_01,201301,zone_a
...,...,...,...,...,...,...,...,...,...
255084,facility_10,846003,small,cod,5,v1,facility_10,846003,zone_c
255085,facility_10,848101,small,prepaid,5,v1,facility_10,848101,zone_c
255086,facility_10,848101,small,prepaid,5,v1,facility_10,848101,zone_c
255087,facility_10,851112,small,cod,5,v1,facility_10,851112,zone_c


In [110]:
# Q-02
# convert file 2 to long format
file02_long = pd.melt(file02_df, id_vars=['zone', 'mode', 'size'], value_vars=['cost_with_vendor_v1', 'cost_with_vendor_v2',
       'cost_with_vendor_v3', 'cost_with_vendor_v4'], value_name='price', var_name='vendor')
file02_long['vendor'] = file02_long['vendor'].str.replace('cost_with_vendor_', '')
file02_long['mode'] = file02_long['mode'].str.replace('COD', 'cod')
file02_long['zone'] = file02_long['zone'].str.replace('zone_C', 'zone_c')
file02_long['zone'] = file02_long['zone'].str.replace('Zone_B', 'zone_b')

# join file 1 with file 2 and 4 to get zone and price information
file01_w_zone = file01_df.merge(file04_df, left_on=['supplier_location', 'customer_location'], \
                                right_on=['source', 'destination'], how='left')
cost_df = file01_w_zone.merge(file02_long, left_on=['shipment_zone', 'payment_option', 'shpment_size', 'current_logistics_vendor'], \
                    right_on=['zone', 'mode', 'size', 'vendor'], how='left')

cost_df = cost_df[['supplier_location', 'customer_location', 'shipment_zone', 'shpment_size',
       'payment_option', 'current_logistics_vendor', 'product_quantity','price']]
cost_df['cost'] = cost_df['product_quantity']*cost_df['price']
cost_df_dedup = cost_df.drop_duplicates()


# pivot the dataframe to get cost per current vendor
q02_pivot = pd.pivot_table(cost_df_dedup, index=['current_logistics_vendor'], values='cost', aggfunc=np.sum)
q02_pivot['% share'] = q02_pivot['cost']*100/q02_pivot['cost'].sum()
print('Total cost of supply chain with current logistic vendors: ', cost_df_dedup['cost'].sum())
display(q02_pivot)

Total cost of supply chain with current logistic vendors:  415029890


,cost,% share
current_logistics_vendor,,
v1,370038040,89.159371
v2,12276540,2.957989
v3,12715930,3.063859
v4,19999380,4.818781


In [114]:
# Q-03

least_price_vendors = file02_long.loc[file02_long.groupby(['zone', 'mode', 'size']).price.idxmin()].reset_index(drop=True)
least_cost_df = file01_w_zone.merge(least_price_vendors, left_on=['shipment_zone', 'payment_option', 'shpment_size'], \
                    right_on=['zone', 'mode', 'size'], how='left')
least_cost_dedup_df = least_cost_df.drop_duplicates()
least_cost_dedup_df['cost'] = least_cost_dedup_df['product_quantity']*least_cost_dedup_df['price']
print('Least possible cost: ', least_cost_dedup_df['cost'].sum())

Least possible cost:  349411400


In [115]:
# Q-04

print('Cost saving by choosing least cost option: ', cost_df_dedup['cost'].sum()-least_cost_dedup_df['cost'].sum())

Cost saving by choosing least cost option:  65618490


### Optional question

In [122]:
least_cost_pivot = pd.pivot_table(least_cost_dedup_df, index=['vendor'], values='cost', aggfunc=np.sum)
least_cost_pivot['% share'] = least_cost_pivot['cost']*100/least_cost_pivot['cost'].sum()
print('Cost distribution per vendor with least cost option')
print(least_cost_pivot)

least_cost_pivot_volume = pd.pivot_table(least_cost_dedup_df, index=['vendor'], values='product_quantity', aggfunc=np.sum)
least_cost_pivot_volume['% share'] = least_cost_pivot_volume['product_quantity']*100/least_cost_pivot_volume['product_quantity'].sum()
print('\n\nVolume distribution per vendor with least cost option')
print(least_cost_pivot_volume)


Cost distribution per vendor with least cost option
             cost    % share
vendor                      
v1       64788990  18.542323
v3      252584900  72.288683
v4       32037510   9.168994


Volume distribution per vendor with least cost option
        product_quantity    % share
vendor                             
v1               1168144  33.325336
v3               2017948  57.568925
v4                319181   9.105739


By comparing the above least cost distribution with the current distribution,
* vendor v3 offer the lowest cost for most of products
* Current distribution of vendors is biased towards vendor v1 even though vendor v3 is the most economical choice
* vendor v2 doesn't offer least price for any products
* In the least cost option, 72% of total cost is paid to vendor v3 but it only has the 57% of total volume. So, vendor v3 offer lowest price among other vendors for high priced products.